Multi-Driver Grand Prix Telemetry Analysis

This notebook engineers telemetry data from the Fast-F1 API for telemetry dataframes for multiple drivers. It utilizes a structured workflow for data acquisition, preparation, and further exploration prior to applying machine learning models to multi-driver behavior analysis results. Additional descriptive statistics are added to single driver data frames that will then be concatenated and used as input for machine learning models.

NOTE: This notebook will not compile as standalone. Please pull down the project from GitHub (git clone https://github.com/yourusername/f1-driving-style-analytics-tool.git) and install the necessary dependencies. Further instruction is included in the README.md

The code below adds the parent directory to Python’s module search path and configures logging to suppress all FastF1 logs below the warning level. This will enable subsequent code blocks that use imports to work seamlessly and keep my resulting code compilations clean and easy to read.

In [1]:
import sys
import os
import logging

root = os.path.abspath("..")
sys.path.append(root)

logging.getLogger('fastf1').setLevel(logging.ERROR)

In this section, I import Python libraries for data visualization, numerical analysis, and working with the Pandas dataframes that the FastF1 API is primarily structured with. I also import custom functions and modules for preprocessing F1 data and constants. To support full visibility into the datasets without truncation, I configure Pandas display options to show all rows and columns.

In [2]:
from src.data import f1_data
from src.utils import f1_constants
from src.preprocessing import telemetry_cleaning, telemetry_processing
from src.viz import plots

import pandas as pd

pd.set_option('display.max_rows', None) # reset_option to compact dataframe view
pd.set_option('display.max_columns', None)

The following code initializes a single F1 race session by defining parameters such as year, location, and session type. These values are passed into the custom F1Session class (from f1_data.py), which creates a session object built on top of Fast-F1. This object provides access to race data as well as custom functions I’ve implemented.

These session parameters were considered to best match Tier 2 controls:

- Location: Bahrain (clear, windy conditions)
- Green Flag Laps: Remove safety car and VSC distortions
- Single-compound Windows: Avoid mixed group comparisons
- Minimize Outliers: Tire age approx. 20 laps
- Traffic: Avoid slipstream bias (lead >2s ahead)
- Corner Analysis → Isolates driver techniques

In [3]:
year = 2025
grand_prix = f1_constants.F1Constants.LOCATIONS["Bahrain"]
session_type = f1_constants.F1Constants.SESSIONS["R"]

safety_car_laps = [32, 33, 34, 35]

session = f1_data.F1Session(year, grand_prix, session_type)

req         WARNING 	DEFAULT CACHE ENABLED! (1.43 GB) /Users/phillipsmith/Library/Caches/fastf1
core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['81', '63', '4', '16', '44', '1', '10', '31', '22', '87', '12', '23', '6', '7', '1

All drivers who participated in the specific location's Grand Prix will be analyzed and assigned variables to be identified by their three-letter name code.

Constants for telemetry data is also initialized as variables for ease of use during visualization.

Sector timestamps variables are set for easy replacment during telemetry filtering.

In [4]:
# driver constants
piastri = f1_constants.F1Constants.DRIVERS["Oscar Piastri"]
russell = f1_constants.F1Constants.DRIVERS["George Russell"]
norris = f1_constants.F1Constants.DRIVERS["Lando Norris"]
leclerc = f1_constants.F1Constants.DRIVERS["Charles Leclerc"]
hamilton = f1_constants.F1Constants.DRIVERS["Lewis Hamilton"]
verstappen = f1_constants.F1Constants.DRIVERS["Max Verstappen"]
gasly = f1_constants.F1Constants.DRIVERS["Pierre Gasly"]
ocon = f1_constants.F1Constants.DRIVERS["Esteban Ocon"]
tsunoda = f1_constants.F1Constants.DRIVERS["Yuki Tsunoda"]
bearman = f1_constants.F1Constants.DRIVERS["Oliver Bearman"]
antonelli = f1_constants.F1Constants.DRIVERS["Andrea Kimi Antonelli"]
albon = f1_constants.F1Constants.DRIVERS["Alexander Albon"]
hadjar = f1_constants.F1Constants.DRIVERS["Isack Hadjar"]
doohan = f1_constants.F1Constants.DRIVERS["Jack Doohan"]
alonso = f1_constants.F1Constants.DRIVERS["Fernando Alonso"]
lawson = f1_constants.F1Constants.DRIVERS["Liam Lawson"]
stroll = f1_constants.F1Constants.DRIVERS["Lance Stroll"]
bortoleto = f1_constants.F1Constants.DRIVERS["Gabriel Bortoleto"]
sainz = f1_constants.F1Constants.DRIVERS["Carlos Sainz"]
hulkenberg = f1_constants.F1Constants.DRIVERS["Nico Hulkenberg"]

# telemetry constants
speed = f1_constants.F1Constants.TELEMETRY_COLUMNS["Speed (m/s)"]
acceleration = f1_constants.F1Constants.TELEMETRY_COLUMNS["Acceleration (m/s²)"]
jerk = f1_constants.F1Constants.TELEMETRY_COLUMNS["Jerk (m/s³)"]
rpm = f1_constants.F1Constants.TELEMETRY_COLUMNS["RPM"]
g_force = f1_constants.F1Constants.TELEMETRY_COLUMNS["G-force (g)"]
gear = f1_constants.F1Constants.TELEMETRY_COLUMNS["nGear"]
throttle = f1_constants.F1Constants.TELEMETRY_COLUMNS["Throttle (%)"]
brakes = f1_constants.F1Constants.TELEMETRY_COLUMNS["BrakesApplied"]
# steering = f1_constants.F1Constants.TELEMETRY_COLUMNS["Steering Wheel Angle (°)"]

# sector timestamp variables
s1_start = 'Sector1Start'
s1_end_s2_start = 'Sector1End_Sector2Start'
s2_end_s3_start = 'Sector2End_Sector3Start'
s3_end = 'Sector3End'

The code below retrieves circuit information for the session’s corners. The `critical_turn` array identifies corners where driver behavior is especially important for sector and overall lap performance. Adjusting the `radius` value controls how much telemetry around each turn is included, allowing you to visualize throttle and braking traces at the desired level of detail.

In [5]:
corner_position = session.get_circuit_info().corners
corner_position_cleaned = telemetry_cleaning.clean_circuit_corner_data(corner_position)

# 10 --> 2500
critical_turn = [10]
radius = 50000

Gather, clean, and enrich each drivers' list of telemetry dataframes for their valid laps.

Each telemetry dataframe is filtered by turn to focus on the most informative segments for analyzing driver behavior and technique at brake-intensive areas.

Lando Norris Race Telemetry & Performance Metrics

In [6]:
norris_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=norris,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Lando Norris Fastest Lap Telemetry for Visualization

In [7]:
norris_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=norris_processed_data,
    driver_code=norris,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Oscar Piastri Race Telemetry & Performance Metrics

In [8]:
piastri_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=piastri,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Oscar Piastri Fastest Lap Telemetry for Visualization

In [9]:
piastri_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=piastri_processed_data,
    driver_code=piastri,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

George Russell Race Telemetry & Performance Metrics

In [10]:
russell_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=russell,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

George Russell Fastest Lap Telemetry for Visualization

In [11]:
russell_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=russell_processed_data,
    driver_code=russell,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Charles Leclerc Race Telemetry & Performance Metrics

In [12]:
leclerc_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=leclerc,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Charles Leclerc Fastest Lap Telemetry for Visualization

In [13]:
leclerc_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=leclerc_processed_data,
    driver_code=leclerc,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Lewis Hamilton Race Telemetry & Performance Metrics

In [14]:
hamilton_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=hamilton,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Lewis Hamilton Fastest Lap Telemetry for Visualization

In [15]:
hamilton_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=hamilton_processed_data,
    driver_code=hamilton,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Max Verstappen Race Telemetry & Performance Metrics

In [16]:
verstappen_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=verstappen,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Max Verstappen Fastest Lap Telemetry for Visualization

In [17]:
verstappen_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=verstappen_processed_data,
    driver_code=verstappen,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Pierre Gasly Race Telemetry & Performance Metrics

In [18]:
gasly_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=gasly,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Pierre Gasly Fastest Lap Telemetry for Visualization

In [19]:
gasly_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=gasly_processed_data,
    driver_code=gasly,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Esteban Ocon Race Telemetry & Performance Metrics

In [20]:
ocon_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=ocon,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Esteban Ocon Fastest Lap Telemetry for Visualization

In [21]:
ocon_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=ocon_processed_data,
    driver_code=ocon,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Yuki Tsunoda Race Telemetry & Performance Metrics

In [22]:
tsunoda_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=tsunoda,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Yuki Tsunoda Fastest Lap Telemetry for Visualization

In [23]:
tsunoda_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=tsunoda_processed_data,
    driver_code=tsunoda,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Oliver Bearman Race Telemetry & Performance Metrics

In [ ]:
bearman_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=bearman,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Oliver Bearman Fastest Lap Telemetry for Visualization

In [ ]:
bearman_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=bearman_processed_data,
    driver_code=bearman,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Andrea Kimi Antonelli Race Telemetry & Performance Metrics

In [ ]:
antonelli_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=antonelli,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Andrea Kimi Antonelli Fastest Lap Telemetry for Visualization

In [ ]:
antonelli_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=antonelli_processed_data,
    driver_code=antonelli,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Alexander Albon Race Telemetry & Performance Metrics

In [ ]:
albon_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=albon,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Alexander Albon Fastest Lap Telemetry for Visualization

In [ ]:
albon_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=albon_processed_data,
    driver_code=albon,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Isack Hadjar Race Telemetry & Performance Metrics

In [ ]:
hadjar_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=hadjar,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Isack Hadjar Fastest Lap Telemetry for Visualization

In [ ]:
hadjar_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=hadjar_processed_data,
    driver_code=hadjar,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Jack Doohan Race Telemetry & Performance Metrics

In [ ]:
doohan_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=doohan,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Jack Doohan Fastest Lap Telemetry for Visualization

In [ ]:
doohan_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=doohan_processed_data,
    driver_code=doohan,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Fernando Alonso Race Telemetry & Performance Metrics

In [ ]:
alonso_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=alonso,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Fernando Alonso Fastest Lap Telemetry for Visualization

In [ ]:
alonso_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=alonso_processed_data,
    driver_code=alonso,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Liam Lawson Race Telemetry & Performance Metrics

In [ ]:
lawson_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=lawson,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Liam Lawson Fastest Lap Telemetry for Visualization

In [ ]:
lawson_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=lawson_processed_data,
    driver_code=lawson,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Lance Stroll Race Telemetry & Performance Metrics

In [ ]:
stroll_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=stroll,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Lance Stroll Fastest Lap Telemetry for Visualization

In [ ]:
stroll_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=stroll_processed_data,
    driver_code=stroll,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Gabriel Bortoleto Race Telemetry & Performance Metrics

In [ ]:
bortoleto_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=bortoleto,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Gabriel Bortoleto Fastest Lap Telemetry for Visualization

In [ ]:
bortoleto_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=bortoleto_processed_data,
    driver_code=bortoleto,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Carlos Sainz Race Telemetry & Performance Metrics

In [ ]:
sainz_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=sainz,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Carlos Sainz Fastest Lap Telemetry for Visualization

In [ ]:
sainz_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=sainz_processed_data,
    driver_code=sainz,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Nico Hulkenberg Race Telemetry & Performance Metrics

In [ ]:
hulkenberg_processed_data = telemetry_processing.process_driver_telemetry(
    session=session,
    driver=hulkenberg,
    safety_car_laps=safety_car_laps,
    corner_position_cleaned=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Nico Hulkenberg Fastest Lap Telemetry for Visualization

In [ ]:
hulkenberg_corner_telemetry_enriched = telemetry_processing.get_fastest_lap_telemetry(
    processed_driver_data=hulkenberg_processed_data,
    driver_code=hulkenberg,
    corner_position=corner_position_cleaned,
    critical_turn=critical_turn[0],
    radius=radius,
    start=s1_end_s2_start,
    end=s2_end_s3_start
)

Visualizations - Multiple Drivers - This multi-axis plot can dynamically display telemetry data to reveal unique driver performance for given driver dataframes.
Limit the number of features and drivers to maintain clarity.

Plot all listed drivers' fastest lap telemetry for a given corner.

In [ ]:
plots.plot_multiple_drivers_telemetry(
    dfs=[
        piastri_corner_telemetry_enriched,
        russell_corner_telemetry_enriched,
        norris_corner_telemetry_enriched,
        leclerc_corner_telemetry_enriched,
        hamilton_corner_telemetry_enriched,
        verstappen_corner_telemetry_enriched,
        gasly_corner_telemetry_enriched,
        ocon_corner_telemetry_enriched,
        tsunoda_corner_telemetry_enriched,
        bearman_corner_telemetry_enriched,
        antonelli_corner_telemetry_enriched,
        albon_corner_telemetry_enriched,
        hadjar_corner_telemetry_enriched,
        doohan_corner_telemetry_enriched,
        alonso_corner_telemetry_enriched,
        lawson_corner_telemetry_enriched,
        stroll_corner_telemetry_enriched,
        bortoleto_corner_telemetry_enriched,
        sainz_corner_telemetry_enriched,
        hulkenberg_corner_telemetry_enriched
    ],
    drivers=[
        piastri, russell, norris, leclerc, hamilton, verstappen, gasly, ocon, tsunoda, bearman, 
        antonelli, albon, hadjar, doohan, alonso, lawson, stroll, bortoleto, sainz, hulkenberg
    ],

    # speed, acceleration, rpm, g_force, gear, throttle, brakes
    telemetry_cols=[brakes]
)

Grid Performance Metrics - All Drivers - The following code concatenates performance metrics for each driver into a single DataFrame.

In [ ]:
all_driver_final_features = pd.concat([
    piastri_processed_data[0],
    russell_processed_data[0],
    norris_processed_data[0],
    leclerc_processed_data[0],
    hamilton_processed_data[0],
    verstappen_processed_data[0],
    gasly_processed_data[0],
    ocon_processed_data[0],
    tsunoda_processed_data[0],
    bearman_processed_data[0],
    antonelli_processed_data[0],
    albon_processed_data[0],
    hadjar_processed_data[0],
    doohan_processed_data[0],
    alonso_processed_data[0],
    lawson_processed_data[0],
    stroll_processed_data[0],
    bortoleto_processed_data[0],
    sainz_processed_data[0],
    hulkenberg_processed_data[0],
], ignore_index=True)

Export the all_driver_final_features dataframe as a Pickle file.

In [ ]:
os.makedirs('exports/final_features', exist_ok=True)

all_driver_final_features.to_pickle('exports/final_features/2025_bahrain_sector2_grandprix.pkl')

The following code:
- Loads a clustered telemetry dataframe
- Removes noise points
- Sorts by a selected feature
- Computes a global z-score to compare each driver record to the group
- Returns key columns for further analysis

NOTE: HDBSCAN must be executed to generate file prior to importing. It is also advised to only display columns that were used as input into the clustering model.

In [ ]:
file_path = 'exports/clustered_dfs/2025_bahrain_sector2_grandprix_hdbscan_clustered.pkl'
df_clustered = pd.read_pickle(file_path)

# remove noise points (cluster -1)
df_filtered = df_clustered[df_clustered['Cluster'] != -1]

# sort by selected label descending
df_clustered_sorted = df_filtered.sort_values(by=['ExitSpeed'], ascending=False)
df_clustered_sorted = df_clustered_sorted.reset_index(drop=True)

# calculate z-score for ExitSpeed
mu = df_clustered_sorted['ExitSpeed'].mean()
sigma = df_clustered_sorted['ExitSpeed'].std()
df_clustered_sorted['Z-Score'] = (df_clustered_sorted['ExitSpeed'] - mu) / sigma

# create separate dataframes for each cluster
cluster_0 = df_clustered_sorted[(df_clustered_sorted['Cluster'] == 0) & (df_clustered_sorted['Z-Score'] >= 0)]
cluster_1 = df_clustered_sorted[(df_clustered_sorted['Cluster'] == 1) & (df_clustered_sorted['Z-Score'] >= 0)]
cluster_2 = df_clustered_sorted[(df_clustered_sorted['Cluster'] == 2) & (df_clustered_sorted['Z-Score'] >= 0)]
cluster_3 = df_clustered_sorted[(df_clustered_sorted['Cluster'] == 3) & (df_clustered_sorted['Z-Score'] >= 0)]
cluster_4 = df_clustered_sorted[(df_clustered_sorted['Cluster'] == 4) & (df_clustered_sorted['Z-Score'] >= 0)]

# calculate and print mean and std for each cluster
cluster_0['InitialBrakeTime'].mean(), cluster_0['BrakeDuration'].mean(), cluster_0['ExitSpeed'].mean(), cluster_0['ExitSpeed'].std(), cluster_0['ExitSpeed'].max()
cluster_1['InitialBrakeTime'].mean(), cluster_1['BrakeDuration'].mean(), cluster_1['ExitSpeed'].mean(), cluster_1['ExitSpeed'].std(), cluster_1['ExitSpeed'].max()
cluster_2['InitialBrakeTime'].mean(), cluster_2['BrakeDuration'].mean(), cluster_2['ExitSpeed'].mean(), cluster_2['ExitSpeed'].std(), cluster_2['ExitSpeed'].max()
cluster_3['InitialBrakeTime'].mean(), cluster_3['BrakeDuration'].mean(), cluster_3['ExitSpeed'].mean(), cluster_3['ExitSpeed'].std(), cluster_3['ExitSpeed'].max()


The following code:
- Calculates noise and clustered data point counts
- Plots the cluster distribution

NOTE: HDBSCAN must be executed to generate file prior to importing. It is also advised to only display columns that were used as input into the clustering model.

In [ ]:
# calculate noise and clustered counts
df_noise = df_clustered[df_clustered['Cluster'] == -1]
noise_count = len(df_noise)
clustered_count = len(df_clustered) - noise_count

# plot cluster distribution
plots.plot_cluster_distribution(
    df_clustered=df_clustered,
    title=f'{year} {grand_prix} {session_type} Turn {critical_turn[0]} \n Clustered Points: {clustered_count} - Noise Points: {noise_count}'
)

Key Research Questions:
- Is there a driving style that is faster overall?
- Which style yields higher average speed?
- Can tire wear be correlated to a driving style? (Not yet explored in this project)

Updated Hypotheses:
- Faster corner exits translate to greater straight-line speed and reduced lap times.
- Drivers who re-engage full throttle at the optimal moment after the apex tend to achieve higher corner exit speeds.
- Drivers who brake precisely at the optimal point maintain greater stability, enabling higher corner exit speeds.
- Aggressive, late breakers are more likely to lose stability through an apex, leading to slower corner exit speeds.
- Drivers who apply fewer braking events through a sequence of turns tend to sustain higher RPM levels, resulting in faster corner exit speeds.

New engineered features:
- Steering Wheel Angle (will be removed as more research is required)
- Initial Brake Timestamp
- Max Brake Duration
- Max Throttle Ramp Initial
- Max Throttle Ramp Time
- Speed at Full Throttle
- Speed at Corner Exit
- Speed Minimum During Turn
- Speed Min to Exit Max
- Turn Duration

Summary
- Clusters weighted toward maximum exit speeds indicate the optimal feature values for a given driving profile.
- HDBSCAN parameters, such as min_cluster_size and min_samples, should be tuned to maximize cluster stability and reduce noise.
- Z-scores >0 indicate that a driver’s features more strongly align with the characteristics of the dependent variable, =0 average, <0 unfavorable.
- Increased volume and quality of input data by using race session over qualifying; insights produced by ml models should lead to more accurate predictions.
- Steering wheel angle was not included in the clustering, yet it could provide significant insights into driver behavior, including oversteer and understeer tendencies.